In [ ]:
import keras  # Keras är ett API för att bygga och träna neurala nätverk, som ofta används med TensorFlow som backend.
from keras import layers  # layers används för att bygga de olika delarna (lager) i ett neuralt nätverk, t.ex. Dense- och Convolutional-lager.
import gymnasium as gym  # Gymnasium är en plattform för att skapa miljöer för reinforcement learning (RL).
from gymnasium.wrappers.frame_stack import FrameStack  
# FrameStack är en wrapper från Gymnasium som kombinerar flera på varandra följande frames till en enda observation.
# Detta är användbart i spel där kontext från tidigare frames behövs, exempelvis rörelseriktning i Atari-spel.

from gymnasium.wrappers.atari_preprocessing import AtariPreprocessing  
# AtariPreprocessing är en inbyggd funktion för att förbehandla bilder från Atari-spel.
# Den konverterar bilder till gråskala, minskar deras storlek och beskär dem för att underlätta inlärning.

import numpy as np  # NumPy används för numeriska operationer som matriser, slumpval och matematiska beräkningar.
import tensorflow as tf  # TensorFlow används som backend för att implementera och träna neurala nätverk.
import ale_py  # ale_py är en wrapper för Arcade Learning Environment (ALE) som tillhandahåller Atari-spel till Gymnasium.

model = keras.models.load_model("models/breakout_qmodel_547.keras")
model_target = keras.models.load_model("models/breakout_qmodel_547.keras")


# Hyperparametrar för RL-träningen
seed = 42  # Slumpfrö för att göra experimenten reproducerbara.
gamma = 0.99  # Diskonteringsfaktor för framtida belöningar. Värden nära 1 prioriterar långsiktiga belöningar.
epsilon = 1.0  # Startvärde för epsilon i epsilon-greedy-algoritmen, som används för att utforska.
epsilon_min = 0.1  # Minsta värde för epsilon, vilket begränsar slumpmässiga val under utforskning.
epsilon_max = 1.0  # Maxvärde för epsilon, används vid start.
epsilon_interval = (epsilon_max - epsilon_min)  
# Intervall för att gradvis minska epsilon under träning, från maximal utforskning till kontrollerad utforskning.

batch_size = 32  # Antal exempel som används vid varje träningssteg.
max_steps_per_episode = 10000  # Max antal steg i en episod innan den avslutas.
max_episodes = 0  # Max antal episoder (0 innebär obegränsat).
max_frames = 1e7  # Max antal bildramar som ska bearbetas innan träningen avslutas.


# Registrerar Atari-miljöer via ALE.
gym.register_envs(ale_py)
# Skapar Atari Breakout-miljön.
env = gym.make("SpaceInvadersNoFrameskip-v4", render_mode="rgb_array")  
# Spelmiljö: Breakout utan att hoppa över några frames. render_mode="rgb_array" gör att vi får bilddata.
env = AtariPreprocessing(env)  

# Applicerar förbehandling på miljön. Detta inkluderar:
# - Omvandling till gråskala
# - Bildskalning till 84x84 pixlar
# - Normalisering av pixlar.

env = FrameStack(env, 4)  
# Staplar fyra på varandra följande frames för att ge modellen temporal kontext (rörelseinformation).

# Triggerfunktion för att spela in video var 1000:e timsteg.
trigger = lambda t: t % 500 == 0
env = gym.wrappers.RecordVideo(env, video_folder="videos", episode_trigger=trigger, disable_logger=True)  
# Wrapper för att spela in videos av episoder som uppfyller trigger-funktionen.

num_actions = env.action_space.n
  # Antal möjliga handlingar i Breakout (t.ex. vänster, höger, stanna, skjut).
  
action_meanings = env.unwrapped.get_action_meanings()
print(f"{num_actions} {action_meanings}") # 'NOOP', 'FIRE', 'RIGHT', 'LEFT', 'RIGHTFIRE', 'LEFTFIRE'

def preprocess_input(data):
    return np.transpose(data, (1, 2, 0))
  
  
# Skapa den primära modellen och en target-modell (för stabilitet vid uppdateringar).
# model = create_q_model()
# model_target = create_q_model()

# Optimizer för träning: Adam med en lärhastighet på 0.00025 och gradientklippning för stabilitet.
optimizer = keras.optimizers.Adam(learning_rate=0.00025, clipnorm=1.0)

# Buffertar och andra variabler för att lagra träningens historia.
action_history = []  # Historik av valda åtgärder.
state_history = []  # Historik av observerade tillstånd.
state_next_history = []  # Historik av efterföljande tillstånd.
rewards_history = []  # Historik av erhållna belöningar.
done_history = []  # Historik av episod-slutstatus.
episode_reward_history = []  # Belöningar för varje avslutad episod.
running_reward = 0  # Medelbelöning över senaste episoderna.
episode_count = 550  # Antal episoder hittills.
frame_count = 280000  # Totalt antal processade frames.280 000

# Parametrar för epsilon-greedy policy.
epsilon_random_frames = 50000  # Antal frames där endast slumpmässiga åtgärder används.
epsilon_greedy_frames = 1000000.0  # Antal frames över vilka epsilon avtar från 1.0 till 0.1.

# Maximal replay buffer-storlek och träningskonfiguration.
max_memory_length = 1000000  # Max antal lagrade övergångar i replay-buffer.
update_after_actions = 4  # Antal actions mellan varje träningsuppdatering.
update_target_network = 10000  # Frekvens (i frames) för att uppdatera target-modellen.
loss_function = keras.losses.Huber()  # Huberförlust, robust för utliggare.


while True:  # En oändlig loop som tränar modellen tills ett avslutande villkor nås.
    observation, _ = env.reset()  # Nollställer miljön och hämtar den initiala observationen.
    state = np.array(observation)  # Konverterar observationen till en NumPy-array.
    episode_reward = 0  # Initierar episodens totala belöning.

    for timestep in range(1, max_steps_per_episode):  # Begränsar antalet steg per episod.
        frame_count += 1  # Ökar den totala bildramräknaren (används för tidsstyrda åtgärder).

        # Epsilon-greedy policy: Välj antingen en slumpmässig åtgärd eller den bästa åtgärden från modellen.
        if frame_count < epsilon_random_frames or epsilon > np.random.rand(1)[0]:
            # Utforskar genom att ta en slumpmässig åtgärd.
            action = np.random.choice(num_actions)
        else:
            # Uppdatera epsilon-greedy steget
            state_processed = preprocess_input(state) #------------------------------------------------
            state_tensor = keras.ops.convert_to_tensor(state_processed)
            state_tensor = keras.ops.expand_dims(state_tensor, 0)
            action_probs = model(state_tensor, training=False)
            # # Exploaterar genom att använda modellen för att välja den bästa åtgärden.
            # state_tensor = keras.ops.convert_to_tensor(state)  # Konverterar tillståndet till en tensor.
            # state_tensor = keras.ops.expand_dims(state_tensor, 0)  # Lägger till batch-dimension.
            # action_probs = model(state_tensor, training=False)  # Förutspår Q-värden utan att aktivera träningsläge.
            # action = keras.ops.argmax(action_probs[0]).numpy()  # Väljer åtgärden med högst Q-värde.

        # Minskar epsilon linjärt för att gradvis minska sannolikheten för slumpmässiga åtgärder.
        epsilon -= epsilon_interval / epsilon_greedy_frames
        epsilon = max(epsilon, epsilon_min)  # Begränsar epsilon till minimumvärdet.

        # Utför åtgärden i miljön och observerar resultatet.
        state_next, reward, done, _, _ = env.step(action)  
        state_next = np.array(state_next)  # Konverterar nästa tillstånd till NumPy-array.

        episode_reward += reward  # Lägg till belöningen från detta steg till episodens totala belöning.

        # Spara övergången i replay-buffer.
        action_history.append(action)
        state_history.append(state)
        state_next_history.append(state_next)
        done_history.append(done)
        rewards_history.append(reward)

        # Gå vidare till nästa tillstånd.
        state = state_next
        
        # Träna modellen efter var fjärde åtgärd och om replay-buffer är tillräckligt stor.
        if frame_count % update_after_actions == 0 and len(done_history) > batch_size:
            # Väljer slumpmässigt ett urval av tidigare övergångar för att skapa träningsdata.
            indices = np.random.choice(range(len(done_history)), size=batch_size)

            # Extraherar de valda övergångarna från replay-buffer.
            state_sample = np.array([state_history[i] for i in indices])
            state_next_sample = np.array([state_next_history[i] for i in indices])
            rewards_sample = [rewards_history[i] for i in indices]
            action_sample = [action_history[i] for i in indices]
            done_sample = keras.ops.convert_to_tensor(
                [float(done_history[i]) for i in indices]
            )

            state_next_sample_processed = np.array([preprocess_input(s) for s in state_next_sample])
            future_rewards = model_target.predict(state_next_sample_processed, verbose=0)

            #future_rewards = model_target.predict(state_next_sample, verbose=0)---------------------------------------------------------
            
                        
            
            # Uppdaterade Q-värden: belöning + gamma * max framtida belöning.
            updated_q_values = rewards_sample + gamma * keras.ops.amax(future_rewards, axis=1)

            # Sätter Q-värdet till -1 om tillståndet är terminalt.
            updated_q_values = updated_q_values * (1 - done_sample) - done_sample

            # Skapar en mask för att beräkna förlust endast för den utförda åtgärden.
            masks = keras.ops.one_hot(action_sample, num_actions)

            with tf.GradientTape() as tape:
                # Förutspår Q-värden för det aktuella tillståndet.
                #q_values = model(state_sample)------------------------------------------------------------------------------------------------
            
            

                state_sample_processed = np.array([preprocess_input(s) for s in state_sample])
                q_values = model(state_sample_processed)


                # Extraherar Q-värdet för den valda åtgärden.
                q_action = keras.ops.sum(keras.ops.multiply(q_values, masks), axis=1)

                # Beräknar förlusten mellan de förväntade och de faktiska Q-värdena.
                loss = loss_function(updated_q_values, q_action)

            # Utför backpropagation för att uppdatera modellens viktparametrar.
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

        # Uppdaterar target-modellen efter ett fast antal frames.
        if frame_count % update_target_network == 0:
            model_target.set_weights(model.get_weights())  # Kopierar vikterna från träningsmodellen.
            print(f"Best score of last 100: {np.max(episode_reward_history)}, running reward: {running_reward:.2f} at episode {episode_count}, frame count {frame_count}")
            model.save(f"models/breakout_qmodel_{episode_count}.keras")  # Sparar modellen efter varje uppdatering.

        # Begränsar replay-buffer till maxlängd genom att ta bort gamla övergångar.
        if len(rewards_history) > max_memory_length:
            del rewards_history[:1]
            del state_history[:1]
            del state_next_history[:1]
            del action_history[:1]
            del done_history[:1]

        # Om episoden är klar, avsluta loopen för denna episod.
        if done:
            break

    # Uppdaterar löpande belöning baserat på de senaste 100 episoderna.
    episode_reward_history.append(episode_reward)
    if len(episode_reward_history) > 100:
        del episode_reward_history[:1]
    running_reward = np.mean(episode_reward_history)

    # Ökar episodräknaren.
    episode_count += 1
    print(f"Episode {episode_count-1}: {episode_reward}")

    # Kontrollera om problemet anses löst baserat på en belöningströskel.
    if running_reward > 800:
        print("Solved at episode {}!".format(episode_count))
        break

    # Avbryter om max episoder eller max frames har nåtts.
    if max_episodes > 0 and episode_count >= max_episodes:
        print("Stopped at episode {}!".format(episode_count))
        break
    if max_frames <= frame_count:
        print(f"Stopped at frame {frame_count}!")
        break